In [1]:
from astropy.coordinates import SkyCoord
import os
import pyvo
import warnings
import urllib
import gzip, shutil, glob
warnings.filterwarnings("ignore", module="astropy.io.votable.*")
warnings.filterwarnings("ignore", module="pyvo.utils.xml.*")

In [2]:
def TwoMass(pos ,size = 0.1, unzip = True ):
    """
    This function simplifies the fits file download process provided by pyvo.

    Parameters
    ----------
    pos : astropy.coordinates.sky_coordinate.SkyCoord
        position information generated by astropy. It could be either from name or Ra and Dec.
    size : float, optional
        The searching cone size, by default 0.1 [deg]
        Note that the size is NOT the image size.
    unzip : bool, optional
        Whether to unzip the downloaded file, by default True.
    """

    # Initiate pyvo service and select 2MASS as our catalogue
    image_service = pyvo.regsearch(servicetype='image', keywords=['2mass'])
    image_table = image_service[0].search(pos=pos, size=size) # index 0 is fixed according to regsearch result
    im_table = image_table.to_table()

    # Preventing from repeating download
    a = im_table.to_pandas()
    b = a[a['format'].str.contains('fits')].drop_duplicates(subset=['band','hem','date','scan','image']) # filter for fits file, exclude html files

    # Start Downloading
    for _, row in b.sort_values(['band','date']).iterrows(): # By sorting to couple JHK bands 
        no = 0

        while os.path.exists(f"{row.band}_{no}.fits.gz"): # file numbering
            no += 1
        urllib.request.urlretrieve(f"{row.download}", f"{row.band}_{no}.fits.gz") # Download images from IRSA
    
    # Since images from IRSA are zipped, we need to unzip them
    if unzip:
        for file in glob.glob("*.gz"):
            with gzip.open(file, 'r') as f_in, open(file.replace(".gz", ""), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)  # unzipped gz files
            os.remove(file)   # delete gz files
    else:
        pass 

In [4]:
TwoMass(SkyCoord.from_name("PTF09gn"))